# Point Cloud Performance

> Download this data from [here](https://ugentbe-my.sharepoint.com/:f:/g/personal/victorlouis_degusseme_ugent_be/EkIZoyySsnZBg56hRq1BqdkBuGlvhAwPWT9HDuqaUB-psA?e=iSehj6) and save the folder in a folder called `data` relative to this notebook.


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
data_dir = os.path.join("data", "competition_sample_0000")

In [ ]:
!ls $data_dir

In [ ]:
import cv2
import numpy as np
import open3d as o3d

from airo_typing import NumpyDepthMapType
from airo_dataset_tools.data_parsers.camera_intrinsics import CameraIntrinsics
from airo_dataset_tools.data_parsers.pose import Pose
from airo_camera_toolkit.point_clouds.conversions import open3d_to_point_cloud, point_cloud_to_open3d
from airo_camera_toolkit.point_clouds.operations import filter_point_cloud, crop_point_cloud


In [ ]:
intrinsics_path = os.path.join(data_dir, "intrinsics.json")
image_left_path = os.path.join(data_dir, "image_left.png")
image_right_path = os.path.join(data_dir, "image_right.png")
depth_map_path = os.path.join(data_dir, "depth_map.tiff")
confidence_map_path = os.path.join(data_dir, "confidence_map.tiff")
point_cloud_path = os.path.join(data_dir, "pointcloud.ply")
camera_pose_path = os.path.join(data_dir, "camera_pose.json")

In [ ]:
with open(intrinsics_path, "r") as f:
    intrinsics_model = CameraIntrinsics.model_validate_json(f.read())
    intrinsics = intrinsics_model.as_matrix()
    resolution = intrinsics_model.image_resolution.as_tuple()

with open(camera_pose_path, "r") as f:
    camera_pose = Pose.model_validate_json(f.read()).as_homogeneous_matrix()

with np.printoptions(precision=3, suppress=True):
    print("Resolution:", resolution)
    print("Intrinsics: \n", intrinsics)
    print("Extrinsics: \n", camera_pose)

In [ ]:
def binarize_confidence(image: NumpyDepthMapType, threshold=50.0):
    confident = image <= threshold
    return confident

depth_map = cv2.imread(depth_map_path, cv2.IMREAD_ANYDEPTH)
confidence_map = cv2.imread(confidence_map_path, cv2.IMREAD_ANYDEPTH)

# Confidence mask
threshold = 1.0  # a value of 1.0 means only the most confidence points will be kept
confidence_binarized = binarize_confidence(confidence_map, threshold)
confidence_mask = confidence_binarized.reshape(-1)

# Bounding box
bbox = (0.35, -0.3, 0.1), (0.7, 0.1, 0.95)
bbox_o3d = o3d.geometry.AxisAlignedBoundingBox(*bbox)

# Open3D pointclouds
pcd_in_camera = o3d.io.read_point_cloud(point_cloud_path)
pcd = pcd_in_camera.transform(camera_pose) # transform to world frame (= base frame of left robot)
pcd_filtered = pcd.select_by_index(np.where(confidence_mask)[0])
pcd_cropped = pcd_filtered.crop(bbox_o3d)

# Airo-mono pointclouds
point_cloud = open3d_to_point_cloud(pcd)
point_cloud_filtered = filter_point_cloud(point_cloud, confidence_mask)
point_cloud_cropped = crop_point_cloud(point_cloud_filtered, bbox)

### Performance comparison
#### Conversion

In [ ]:
%%timeit
point_cloud_to_open3d(point_cloud)

In [ ]:
%%timeit
open3d_to_point_cloud(pcd)

#### Filtering

In [ ]:
%%timeit
filter_point_cloud(point_cloud, confidence_mask)

In [ ]:
%%timeit
pcd.select_by_index(np.where(confidence_mask)[0])

#### Cropping

In [ ]:
%%timeit
crop_point_cloud(point_cloud_filtered, bbox)

In [ ]:
%%timeit
pcd_filtered.crop(bbox_o3d)

#### Downsampling

In [ ]:
%%timeit
pcd_cropped.voxel_down_sample(voxel_size=0.01)